# Business Problem

A bank requested a machine learning model that would produce results on whether loan applicants among the bank's customers would get the loan or not.

Data Quality                                                                                
Descriptive Statistics
Modelling
Evaluation

1. Data Quality
It is the dataset that contains information about customers and we will work on deriving useful insights from the information and predicting whether a new customer will accept a loan offer.

1 - age: The age of the customer, must be over 18(numeric)

2 - job: customer is profession(categorical)

admin.
blue-collar
entrepreneur
housemaid
management
self-employed
retired
services
student
technician
unemployed
unknown
3 - marital: marital status(categorical)

married
single
divorced Note: 'divorced' means divorced or widowed
4 - education: educational status(categorical)

primary
secondary
tertiary
unknown
5 - balance: bank balance

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - campaign: number of contacts performed during this campaign and for this client (numeric)

8 - deposit: did the customer get a loan? (binary: 'yes', 'no')

9 - duration: contact time in seconds (numeric)

Libraries

In [1]:
!pip install lazypredict

In [2]:
import os
import lazypredict
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import matplotlib.pyplot as pyplot
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, auc, roc_curve

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [10]:
def load():
    df = pd.read_csv('data/bank.csv')
    return df

df = load()

df = df.drop(labels = ['default', 'contact', 'day', 'month', 'pdays', 'previous', 'loan', 'poutcome', 'poutcome'], axis=1)

Overview

In [12]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(5))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(11162, 9)
##################### Types #####################
age           int64
job          object
marital      object
education    object
balance       int64
housing      object
duration      int64
campaign      int64
deposit      object
dtype: object
##################### Head #####################
   age         job  marital  education  balance housing  duration  campaign  \
0   59      admin.  married  secondary     2343     yes      1042         1   
1   56      admin.  married  secondary       45      no      1467         1   
2   41  technician  married  secondary     1270     yes      1389         1   
3   55    services  married  secondary     2476     yes       579         1   
4   54      admin.  married   tertiary      184      no       673         2   

  deposit  
0     yes  
1     yes  
2     yes  
3     yes  
4     yes  
##################### NA #####################
age          0
job          0
marital      0
educati